In [1]:
import os
import sys
import re

project_root = "/root/work/tenset"
os.environ["TVM_HOME"] = f"{project_root}"
os.environ["TVM_LIBRARY_PATH"] = f"{project_root}/build"
if f"{project_root}/python" not in sys.path:
    sys.path.insert(0, f"{project_root}/python")
    

sys.path = [p for p in sys.path if not p.startswith(f"{project_root}/build")]
sys.path.append(f"{project_root}/build")
os.environ["LD_LIBRARY_PATH"] = f"{project_root}/build:" + os.environ.get("LD_LIBRARY_PATH", "")

In [2]:
import numpy as np
# 데이터 생성 및 전처리
with open("/root/work/tenset/scripts/example_no_del.txt", "r") as f:
    lines = f.read()

records_raw = lines.strip().split("="*60)[1:]
records = {"schedules": [], "extents": [], "costs": [], "unroll" : [], "feature": []}


for rec in records_raw:
    costs = [float(x) for x in re.findall(r'\d+\.\d+', rec.split("Placeholder")[0])]
    cost = -np.log(np.mean(costs) + 1e-8)
    schedule = rec.split("Placeholder")[-1][2:]
    
    records["schedules"].append(schedule)
    records["costs"].append(cost)
    

In [3]:
sys.path.insert(0, "/root/work/tenset/scripts/pre_experiments")
from extract_i_vectors import process_only_diff
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

json_diffs = process_only_diff("/root/work/tenset/dataset/measure_records_tenset/k80/([0c9a5ba46ffc5e1a9e5641018527117f,4,7,7,160,1,1,160,960,1,1,1,960,4,7,7,960],cuda).json", limit=None)
raw_input = json_diffs["diff_values"]
# cost = -np.log(json_diffs["cost"])



In [4]:
import re
import numpy as np

def find_common_for_loops(schedules):
    """
    모든 스케줄에서 공통으로 나타나는 (0,1) for문 변수명을 찾음
    """
    common_vars = None
    
    for schedule in schedules:
        lines = schedule.split('\n')
        vars_in_schedule = set()
        
        for line in lines:
            stripped = line.lstrip()
            match = re.match(r'for\s+(\S+)\s+\(0,\s*1\)', stripped)
            if match:
                vars_in_schedule.add(match.group(1))
        
        if common_vars is None:
            common_vars = vars_in_schedule
        else:
            common_vars &= vars_in_schedule  # 교집합
    
    return common_vars if common_vars is not None else set()


def remove_common_for_loops(schedule, common_vars):
    """
    스케줄 코드에서 공통으로 나타나는 (0,1) for문을 제거하고 들여쓰기를 정리
    """
    lines = schedule.split('\n')
    result_lines = []
    
    # 제거할 for문의 인덱스들을 먼저 찾기
    remove_indices = set()
    for_loop_indents = {}  # 제거될 for문의 인덱스 -> 들여쓰기 레벨
    
    for i, line in enumerate(lines):
        stripped = line.lstrip()
        indent_level = len(line) - len(stripped)
        
        # (0,1) for문인지 확인
        match = re.match(r'for\s+(\S+)\s+\(0,\s*1\)', stripped)
        if match and match.group(1) in common_vars:
            remove_indices.add(i)
            for_loop_indents[i] = indent_level
    
    # 각 줄에 대해 들여쓰기를 얼마나 줄여야 하는지 계산
    indent_reduction = [0] * len(lines)
    
    for idx in sorted(remove_indices):
        base_indent = for_loop_indents[idx]
        # 이 for문 다음부터 같거나 작은 들여쓰기가 나올 때까지 2칸씩 줄이기
        for j in range(idx + 1, len(lines)):
            if j in remove_indices:
                continue
            line = lines[j]
            stripped = line.lstrip()
            if not stripped:  # 빈 줄
                continue
            current_indent = len(line) - len(stripped)
            
            # 이 for문의 body인 경우 (들여쓰기가 더 큰 경우)
            if current_indent > base_indent:
                indent_reduction[j] += 2
            else:
                # 같거나 작은 들여쓰기 레벨이 나오면 이 for문 블록 종료
                break
    
    # 제거하지 않는 줄들에 대해 들여쓰기를 조정하여 결과 생성
    for i, line in enumerate(lines):
        if i in remove_indices:
            continue
        
        if not line.strip():  # 빈 줄
            result_lines.append(line)
            continue
        
        stripped = line.lstrip()
        original_indent = len(line) - len(stripped)
        new_indent = max(0, original_indent - indent_reduction[i])
        result_lines.append(' ' * new_indent + stripped)
    
    return '\n'.join(result_lines)


common_for_loops = find_common_for_loops(records["schedules"])
print(f"발견된 공통 (0,1) for문 변수: {common_for_loops}")


# 모든 스케줄에 적용
cleaned_schedules = []
records["extents"] = []
records["unroll"] = []
records["all"] = []
for i, schedule in enumerate(records["schedules"]):
    extents = [float(x) for x in re.findall(r'\(0,\s*(\d+)\)', schedule)]

for i, schedule in enumerate(records["schedules"]):
    extents = [float(x) for x in re.findall(r'\(0,\s*(\d+)\)', schedule)]
    unrolls = [float(x) for x in re.findall(r'auto_unroll:\s*(\d+)', schedule)]
    records["extents"].append(extents)
    if unrolls == []:
        unrolls = [0.0]
    records["unroll"].append(unrolls)
    feature = extents+unrolls
    records["all"].append(np.array(feature, dtype=np.float32))
    
    cleaned = remove_common_for_loops(schedule, common_for_loops)
    cleaned_schedules.append(cleaned)
records["cleaned_schedules"] = cleaned_schedules


total_removed = sum(len(orig.split('\n')) - len(clean.split('\n')) 
                    for orig, clean in zip(records['schedules'], cleaned_schedules))
avg_removed = total_removed / len(cleaned_schedules)
print(f"제거된 줄 수: {avg_removed:.1f}")

발견된 공통 (0,1) for문 변수: {'yy_c.0', 'ff_c.1', 'xx_c.1', 'ry.2', 'yy_c.1', 'nn_c.0', 'nn_c.2', 'ff_c.0', 'rx.1', 'rx.0', 'rx.2', 'ry.0', 'nn_c.1', 'ry.1', 'xx_c.2', 'ff_c.2', 'yy_c.2', 'xx_c.0'}
제거된 줄 수: 18.0


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np

class sch_extent_RegressionDataset(Dataset):
    def __init__(self, X, y, extent=None):
        if isinstance(X, np.ndarray):
            self.X = torch.from_numpy(X).float()
        else:
            self.X = X
        self.y = torch.from_numpy(y).float()
        if self.y.ndim == 1:
            self.y = self.y.unsqueeze(1)

        self.extent = extent
        if extent is not None:
            if isinstance(extent, np.ndarray):
                self.extent = torch.from_numpy(extent).float()
            else:
                self.extent = extent
            
            if self.extent.ndim == 1:
                self.extent = self.extent.unsqueeze(1)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        if self.extent is None:
            return self.X[idx], self.y[idx]
        return self.X[idx], self.y[idx], self.extent[idx]


class sch_extent_Dataset(Dataset):
    def __init__(self, X, extent=None):
        if isinstance(X, np.ndarray):
            self.X = torch.from_numpy(X).float()
        else:
            self.X = X
        
        if isinstance(extent, np.ndarray):
            self.extent = torch.from_numpy(extent).float()
        else:
            self.extent = extent
        # extent shape이 (N,)이면 (N,1)로 바꿔주는 게 편할 때가 많음
        if self.extent is not None and self.extent.ndim == 1:
            self.extent = self.extent.unsqueeze(1)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        if self.extent is None:
            return self.X[idx]
        return self.X[idx], self.extent[idx]

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class VAE_bottleneck(nn.Module):
    def __init__(self, input_dim, extent_dim=None, latent_dim=16, hidden_dim=128):
        """
        input_dim: 2 * D (v_norm + is_zero concat한 차원)
        latent_dim: latent space 차원
        hidden_dim: MLP hidden 크기
        """
        super().__init__()

        # Encoder
        self.encoder1 = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            
        )
        self.encoder2 = nn.Sequential(
            nn.Linear(extent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim),
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)


        if extent_dim is None:
            self.use_extent = False
        else:
            self.use_extent = True
            self.extent_predictor = nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, extent_dim),  # extents.shape[1]는 extent 차원
            )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            
            # 출력은 연속값이니까 activation 없이 그대로
        )

        

    def encode(self, x):
        h1 = self.encoder1(x)
        extent_pred = self.extent_predictor(h1)
        h2 = self.encoder2(extent_pred)
        

        mu = self.fc_mu(h2)
        logvar = self.fc_logvar(h2)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def predict_extent(self, z):
        return self.extent_predictor(z)

    def forward(self, x, use_mean=True):
        mu, logvar = self.encode(x)
        if use_mean:
            z = mu
        else:
            z = self.reparameterize(mu, logvar)
        x_recon = self.decode(z)
        
        if self.use_extent:
            extent_pred = self.predict_extent(z)
        else:
            extent_pred = None
        return x_recon, mu, logvar, z, extent_pred
        # return mu, logvar, z, extent_pred

class L3Loss(torch.nn.Module):
    def forward(self, pred, target):
        return torch.mean(torch.abs(pred - target) ** 4)

def vae_feature_loss(x_recon, x, mu, logvar, feature_pred, feature, alpha_recon=0, alpha_feature=0, beta=1.0):
    """
    x, x_recon: (B, input_dim)
    mu, logvar: (B, latent_dim)

    beta: KL 가중치 (β-VAE 스타일로 조절)
    """
    # reconstruction loss: MSE
    recon_loss = F.mse_loss(x_recon, x, reduction="mean")
    # 
    # recon_loss = L3Loss()(x_recon, x)

    feature_loss = F.mse_loss(feature_pred, feature, reduction="mean") if feature_pred is not None else 0.0

    # KL divergence: D_KL(q(z|x) || N(0, I))
    kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

    loss = alpha_recon * recon_loss + beta * kl + alpha_feature * feature_loss
    return loss, recon_loss, kl, feature_loss


def bottleneck_extent_loss(mu, logvar, extent_pred, extent, alpha_feature=1.0, beta=1.0):
    """

    beta: KL 가중치 (β-VAE 스타일로 조절)
    """
    # reconstruction loss: MSE
    # recon_loss = F.mse_loss(x_recon, x, reduction="mean")
    # 
    # recon_loss = L3Loss()(x_recon, x)

    extent_loss = F.mse_loss(extent_pred, extent, reduction="mean") if extent_pred is not None else 0.0

    # KL divergence: D_KL(q(z|x) || N(0, I))
    kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

    loss = beta * kl + alpha_feature * extent_loss
    return loss, kl, extent_loss



In [7]:
def seed_everything(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    

In [8]:
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


extent_data = np.log1p(np.array(records["all"], dtype=np.float32))
sch_data = np.log(json_diffs["diff_values"]+1e-8)

scaler = StandardScaler()
extent_data_scaled = scaler.fit_transform(extent_data)
sch_data_scaled = scaler.fit_transform(sch_data)


sch_train, sch_val, extent_train, extent_val = train_test_split(
    sch_data_scaled, extent_data_scaled,test_size=0.2, random_state=42
)


train_dataset = sch_extent_Dataset(sch_train, extent_train)
val_dataset   = sch_extent_Dataset(sch_val, extent_val)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=512, shuffle=False)


In [9]:
from sklearn.metrics import r2_score
import itertools
import torch
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



input_dim = sch_train.shape[-1]
latent_dim = 64
hidden_dim = 256
extent_dim = extent_train.shape[-1]


hyperparameter = {
    'beta': [0.1],
    'alpha_recon': [1.0],
    'alpha_feature': [1.0],
    'latent_dim': [64],
    'lr': [1e-3],
}

cnt = 0
epochs = 100


for vals in itertools.product(*hyperparameter.values()):
    (beta, alpha_recon, alpha_feature, latent_dim, lr) = vals
    cnt += 1
    print("=============================================")
    print(f"Experiment {cnt}/{len(list(itertools.product(*hyperparameter.values())))}")
    print(f"beta={beta}, alpha_recon={alpha_recon}, alpha_feature={alpha_feature},\nepochs={epochs}, latent_dim={latent_dim}, hidden_dim={hidden_dim}, lr={lr}")

    seed_everything(42)

    vae = VAE_feature_head(input_dim=input_dim, extent_dim=extent_dim, latent_dim=latent_dim, hidden_dim=hidden_dim).to(device)
    # decoder freeze
    for param in vae.decoder.parameters():
        param.requires_grad = False
    optimizer = torch.optim.Adam(vae.parameters(), lr=lr)


    # early stopping
    best_val_loss = float('inf')
    patience = 30
    patience_counter = 0

    for epoch in range(1, epochs+1):
        vae.train()
        for x_batch in train_loader:
            if len(x_batch) == 2:
                x_batch, extent_batch = x_batch
                extent_batch = extent_batch.to(device)
            else:
                extent_batch = None
            x_batch = x_batch.to(device)  # (N, D)
            
            

            mu, logvar, z, extent_pred = vae(x_batch, use_mean=False)

            loss, kl, extent_loss = bottleneck_extent_loss(mu, logvar, extent_pred, extent_batch, beta=beta)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        vae.eval()
        for x_batch in val_loader:
            if len(x_batch) == 2:
                x_batch, extent_batch = x_batch
                extent_batch = extent_batch.to(device)
            else:
                extent_batch = None
            x_batch = x_batch.to(device)
            if extent_batch is not None:
                extent_batch = extent_batch.to(device)
            mu, logvar, z, extent_pred = vae(x_batch, use_mean=True)
            val_loss, val_kl, val_extent_loss = bottleneck_extent_loss(mu, logvar, extent_pred, extent_batch, beta=beta)
            # val_recon_r2 = r2_score(x_batch.detach().cpu().numpy(), x_recon.detach().cpu().numpy())
            if extent_batch is not None:
                val_extent_r2 = r2_score(extent_batch.detach().cpu().numpy(), extent_pred.detach().cpu().numpy())
            else:
                val_extent_r2 = None
            if epoch % 20 == 0:
                print(f"epoch {epoch}: val extent R2={val_extent_r2}")
                print(f"epoch {epoch}: val extent loss={val_extent_loss.item():.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss.item()
            patience_counter = 0
        else:
            patience_counter += 1
            
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
    # print(f"epoch {epoch}: loss={loss.item():.4f}, recon={recon_loss.item():.4f}, kl={kl.item():.4f}")
    # print(f"epoch {epoch}: val loss={val_loss.item():.4f}, val recon={val_recon_loss.item():.4f}, val kl={val_kl.item():.4f}")
    print(f"epoch {epoch}: loss={loss.item():.4f}, kl={kl.item():.4f}")
    print(f"epoch {epoch}: val loss={val_loss.item():.4f}, val kl={val_kl.item():.4f}")

    print(f"Extent R2 : {val_extent_r2}")


Experiment 1/1
beta=0.1, alpha_recon=1.0, alpha_feature=1.0,
epochs=100, latent_dim=64, hidden_dim=256, lr=0.001


NameError: name 'VAE_feature_head' is not defined

In [10]:
class VAECostPredictor(nn.Module):
    """
    VAE 기반 Cost Regression 모델
    
    구조:
    - input → segment_encoder → segment_sum → VAE encoder → z → cost_predictor → cost
    
    특징:
    - Pretrained VAE encoder를 finetune (작은 learning rate)
    - Cost predictor는 더 큰 learning rate로 학습
    - 전체 forward 경로가 완전히 미분 가능 (detach, stop_grad 없음)
    """
    
    def __init__(self, input_dim, feature_dim=None, hidden_dim=256, latent_dim=64, 
                 predictor_hidden=256, predictor_layers=2, dropout=0.1, use_feature=False):
        super(VAECostPredictor, self).__init__()
        
        self.input_dim = input_dim
        self.feature_dim = feature_dim
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        
        # ========== Cost Predictor (새로 학습) ==========
        predictor_modules = []
        current_dim = latent_dim
        for i in range(predictor_layers):
            predictor_modules.extend([
                nn.Linear(current_dim, predictor_hidden),
                nn.ReLU(),
                nn.Dropout(dropout) if i < predictor_layers - 1 else nn.Identity(),
            ])
            current_dim = predictor_hidden
        predictor_modules.append(nn.Linear(predictor_hidden, 1))
        
        self.cost_predictor = nn.Sequential(*predictor_modules)

        self.use_feature = use_feature
        if self.use_feature:
            pass
            self.feature_predictor = nn.Sequential(
                nn.Linear(latent_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, feature_dim),  # feature_dim는 feature 차원
            )
        
    
    def encode(self, input_data):
        """
        Full encoding path: features → z
        완전히 미분 가능
        """
                
        # VAE Encoder
        h = self.encoder(input_data)
        
        mean = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        
        return mean, logvar, input_data
    
    def reparameterize(self, mean, logvar):
        """Reparameterization trick - 미분 가능"""
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std
    
    def predict_cost(self, z):
        """z → cost prediction - 완전히 미분 가능"""
        return self.cost_predictor(z).squeeze(-1)
    
    def predict_feature(self, z):
        return self.feature_predictor(z)
    
    def forward(self, input_data, use_mean=True):
        """
        Forward pass: input → z → cost
        
        Args:
            use_mean: True면 reparameterize 대신 mean 사용 (inference용)
        
        Returns:
            cost_pred: 예측된 cost
            mean: latent mean
            logvar: latent log-variance
            z: sampled/mean latent vector
        """
        mean, logvar, input_data = self.encode(input_data)
        
        if use_mean:
            z = mean  # Inference시 deterministic
        else:
            z = self.reparameterize(mean, logvar)  # Training시 stochastic
        
        cost_pred = self.predict_cost(z)
        
        return cost_pred, mean, logvar, z
    
    def get_encoder_params(self):
        """Encoder 파라미터 (작은 lr)"""
        encoder_params = []
        encoder_params.extend(self.encoder.parameters())
        encoder_params.extend(self.fc_mu.parameters())
        encoder_params.extend(self.fc_logvar.parameters())
        return encoder_params
    
    def get_cost_predictor_params(self):
        """Predictor 파라미터 (큰 lr)"""
        return self.cost_predictor.parameters()
    
    def get_feature_predictor_params(self):
        """Feature Predictor 파라미터"""
        return self.feature_predictor.parameters()

    def load_pretrained_encoder(self, checkpoint):
        """Pretrained VAE encoder 가중치 로드"""
        

        vae_state = checkpoint
        
        # 매칭되는 키만 로드
        encoder_keys = ['encoder', 'fc_mu', 'fc_logvar']
        own_state = self.state_dict()
        
        loaded_keys = []
        for name, param in vae_state.items():
            if any(name.startswith(k) for k in encoder_keys):
                if name in own_state and own_state[name].shape == param.shape:
                    own_state[name].copy_(param)
                    loaded_keys.append(name)
        
        # print(f"Loaded {len(loaded_keys)} parameters from pretrained VAE")
        # return loaded_keys

def pair_accuracy(cost_pred, labels):
    """
    cost_pred, labels: (B,) 텐서
    """
    n_samples = min(1000, len(cost_pred))
    sample_indices = np.random.choice(len(cost_pred), n_samples, replace=False)

    with torch.no_grad():
        correct = 0
        total = 0
        for i in range(n_samples):
            for j in range(i + 1, n_samples):
                idx_i = sample_indices[i]
                idx_j = sample_indices[j]
                pred_diff = cost_pred[idx_i] - cost_pred[idx_j]
                true_diff = labels[idx_i] - labels[idx_j]
                if (pred_diff * true_diff) > 0:
                    correct += 1
                total += 1
        accuracy = correct / total if total > 0 else 0.0
    return accuracy


In [11]:
def reg_loss_fn(cost_pred, cost_true, loss_type='mse'):
    """
    기본 회귀 손실 (MSE 또는 MAE)
    """
    if loss_type == 'mse':
        return F.mse_loss(cost_pred, cost_true)
    else:  # mae
        return F.l1_loss(cost_pred, cost_true)


def pair_loss_fn(cost_pred, cost_true, margin=0.1):
    """
    Pairwise ranking loss: 실제 cost 순서를 예측이 유지하도록.
    cost_true[i] < cost_true[j] 이면 cost_pred[i] < cost_pred[j] + margin
    """
    batch_size = cost_pred.size(0)
    if batch_size < 2:
        return torch.tensor(0.0, device=cost_pred.device)
    
    # 모든 쌍에 대해 ranking loss 계산
    idx = torch.arange(batch_size, device=cost_pred.device)
    i_idx, j_idx = torch.meshgrid(idx, idx, indexing='ij')
    mask = i_idx < j_idx  # upper triangular only
    
    pred_i = cost_pred[i_idx[mask]]
    pred_j = cost_pred[j_idx[mask]]
    true_i = cost_true[i_idx[mask]]
    true_j = cost_true[j_idx[mask]]
    
    # label: 1 if true_i < true_j, -1 otherwise
    labels = torch.sign(true_j - true_i).float()
    
    # Margin ranking loss
    loss = F.margin_ranking_loss(pred_j.view(-1), pred_i.view(-1), labels.view(-1), margin=margin)
    return loss


def smooth_loss_fn(model, z, noise_std=0.1):
    """
    Smoothness loss: z에 작은 노이즈를 더했을 때 예측이 크게 변하지 않도록.
    """
    z_noisy = z + noise_std * torch.randn_like(z)
    
    cost_original = model.predict_cost(z)
    cost_noisy = model.predict_cost(z_noisy)
    
    smooth_loss = F.mse_loss(cost_original, cost_noisy)
    return smooth_loss


def kld_loss_fn(mean, logvar):
    """
    KL Divergence: q(z|x) || N(0, I)
    """
    kld = -0.5 * torch.mean(1 + logvar - mean.pow(2) - logvar.exp())
    return kld

def feature_loss_fn(use_feature, feature_pred, feature_true, coef=0.1):
    """
    Feature 예측 손실 (MSE)
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if not use_feature:
        return torch.tensor(0.0, device=device)
    return F.mse_loss(feature_pred, feature_true) * coef


def compute_total_loss(model, cost_pred, mean, logvar, z, labels, feature, config, return_components=True):
    """
    Total loss 계산 (Segment 기반 데이터용).
    total_loss = reg_loss + λ_pair * pair_loss + γ * smooth_loss + β * kld_loss
    """
    
    # Individual losses
    reg = reg_loss_fn(cost_pred, labels, loss_type=config.get('loss_type', 'mse'))
    pair = pair_loss_fn(cost_pred.view(-1), labels.view(-1), margin=config.get('margin', 0.1))
    smooth = smooth_loss_fn(model, z, noise_std=config.get('noise_std', 0.1))
    kld = kld_loss_fn(mean, logvar)
    feature_loss = feature_loss_fn(model.use_feature, None, feature, coef=config.get('alpha', 0.1))
    
    # Weighted sum
    total = config['lambda_reg'] * reg + config['lambda_pair'] * pair + config['gamma'] * smooth + config['beta'] * kld + feature_loss
    
    if return_components:
        return total, {
            'reg_loss': reg.item(),
            'pair_loss': pair.item(),
            'smooth_loss': smooth.item(),
            'kld_loss': kld.item(),
            'feature_loss': feature_loss.item(),
        }
    return total

In [ ]:
costs = np.array(records["costs"], dtype=np.float32)



random_indices = np.random.permutation(len(sch_data_scaled))

train_size = 64

X_train = sch_data_scaled[random_indices[:train_size]]
X_val = sch_data_scaled[random_indices[train_size:]]
y_train = costs[random_indices[:train_size]]
y_val = costs[random_indices[train_size:]]

costs = np.array(records["costs"])

# X_train, X_val, y_train, y_val = train_test_split(
#     input_data_scaled, costs, test_size=0.2, random_state=42
# )

train_dataset = sch_extent_RegressionDataset(X_train, y_train)
val_dataset   = sch_extent_RegressionDataset(X_val,   y_val)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)
val_loader   = DataLoader(val_dataset,   batch_size=512, shuffle=False)



y_mean = y_train.mean()
y_std = y_train.std() + 1e-8  # 0 나누기 방지용 작은 값 추가
print(f"y_train mean: {y_mean}, std: {y_std}")

y_train mean: 6.55483865737915, std: 1.8638334374291992


In [51]:
import itertools
import pandas as pd
input_dim = X_train.shape[-1]
latent_dim = 64
hidden_dim = 256


hyperparameter = {
    # 'alpha': [1e-4, 1e-3],
    
    
    'lambda_reg' : [0.01],
    'lambda_pair': [2.0, 3.0],
    'margin_scale': [0.3],
    'gamma': [0.01],
    'beta': [0.01],
    'noise_std': [0.001],
    'alpha': [1e-4],

    'encoder_lr': [1e-4],
    'feature_predictor_lr': [0],
    'cost_predictor_lr': [1e-2],
    'use_feature': [False],
    # 'seed': [2023, 2024],
    'seed': range(2000, 2010),
    
}



all_reg_results = []
cnt = 0

for vals in itertools.product(*hyperparameter.values()):
    
    
    (lambda_reg, lambda_pair, margin_scale, gamma, beta, noise_std, alpha,
     encoder_lr, feature_predictor_lr, cost_predictor_lr, use_feature, seed) = vals
    
    cnt += 1
    print("=============================================")
    print(f"Experiment {cnt}/{len(list(itertools.product(*hyperparameter.values())))}")
    print(f"lambda_reg={lambda_reg}, lambda_pair={lambda_pair}, margin_scale={margin_scale}, gamma={gamma}, beta={beta}, noise_std={noise_std}, alpha={alpha},\nencoder_lr={encoder_lr}, feature_predictor_lr={feature_predictor_lr}, cost_predictor_lr={cost_predictor_lr}, seed={seed}, use_feature={use_feature}")

    config = {
                'encoder_lr': encoder_lr,
                'feature_predictor_lr': feature_predictor_lr,
                'cost_predictor_lr': cost_predictor_lr,
                'lambda_reg' : lambda_reg,
                'lambda_pair': lambda_pair,
                'gamma': gamma,
                'beta': beta,
                'margin': margin_scale * y_std,
                'noise_std': noise_std,
                'alpha': alpha,
                'loss_type': 'mse'
            }


    torch.manual_seed(seed)

    vae_cost_model = VAECostPredictor(input_dim=input_dim, 
                                    latent_dim=latent_dim, 
                                    hidden_dim=hidden_dim, 
                                    predictor_layers=2,
                                    dropout=0.1, use_feature=use_feature).to(device)
    vae_cost_model.load_pretrained_encoder(vae.state_dict())


    optimizer = torch.optim.AdamW([
            {'params': vae_cost_model.get_encoder_params(), 'lr': config['encoder_lr']},
            # {'params': vae_cost_model.get_feature_predictor_params(), 'lr': config['feature_predictor_lr']},
            {'params': vae_cost_model.get_cost_predictor_params(), 'lr': config['cost_predictor_lr']}
        ], weight_decay=1e-5)


    epochs = 1000
    best_val_loss = float('inf')
    patience = 30
    patience_counter = 0

    for epoch in range(1, epochs+1):
        vae_cost_model.train()
        for x_batch, labels in train_loader:
            x_batch = x_batch.to(device)
            labels = labels.to(device).squeeze(-1)
            
        
            cost_pred, mean, logvar, z = vae_cost_model(x_batch, use_mean=True)

            train_loss, train_components = compute_total_loss(vae_cost_model, 
                                                    cost_pred, mean, logvar, z, labels, None, config)

            optimizer.zero_grad()
            train_loss.backward()
            torch.nn.utils.clip_grad_norm_(vae_cost_model.parameters(), max_norm=1.0)
            optimizer.step()
            
        

        if epoch == epochs:
            vae_cost_model.eval()
            all_preds = []
            all_labels = []
            for x_batch, labels in val_loader:
                x_batch = x_batch.to(device)
                labels = labels.to(device).squeeze(-1)

                cost_pred, mean, logvar, z = vae_cost_model(x_batch, use_mean=True)

                val_loss, val_components = compute_total_loss(vae_cost_model, cost_pred, mean, logvar, z, labels, None, config)
            val_reg_r2 = r2_score(cost_pred.detach().cpu().numpy(), labels.detach().cpu().numpy())
                
            print(f"Train loss epoch {epoch} : reg={train_components['reg_loss']: .4f} rank={train_components['pair_loss']: .4f} kl={train_components['kld_loss']: .4f}")
            print(f"Val loss epoch {epoch}: reg={val_components['reg_loss']: .4f} rank={val_components['pair_loss']: .4f} kl={val_components['kld_loss']: .4f}")
            
            print(f"Regression R2 : {val_reg_r2:.4f}, ", end='')
        
        # rank r2 계산
        if epoch == epochs:
            all_preds = []
            all_labels = []
            for x_batch, labels in val_loader:
                x_batch = x_batch.to(device)
                labels = labels.to(device).squeeze(-1)
                cost_pred, mean, logvar, z = vae_cost_model(x_batch, use_mean=True)
                all_preds.append(cost_pred.detach().cpu().numpy())
                all_labels.append(labels.detach().cpu().numpy())
            all_preds = np.concatenate(all_preds, axis=0)
            all_labels = np.concatenate(all_labels, axis=0)
            val_rank_r2 = pair_accuracy(all_preds, all_labels)
            print(f"Rank R2 : {val_rank_r2:.4f}")
    all_reg_results.append({
        "lambda_reg": lambda_reg,
        "lambda_pair": lambda_pair,
        "margin_scale": margin_scale,
        "gamma": gamma,
        "beta": beta,
        "noise_std": noise_std,
        "alpha": alpha,
        "encoder_lr": encoder_lr,
        "feature_predictor_lr": feature_predictor_lr,
        "cost_predictor_lr": cost_predictor_lr,
        "use_feature": use_feature,
        "seed": seed,
        "reg_r2": val_reg_r2,
        "rank_r2": val_rank_r2,
    })
    

Experiment 1/20
lambda_reg=0.01, lambda_pair=2.0, margin_scale=0.3, gamma=0.01, beta=0.01, noise_std=0.001, alpha=0.0001,
encoder_lr=0.0001, feature_predictor_lr=0, cost_predictor_lr=0.01, seed=2000, use_feature=False
Train loss epoch 1000 : reg= 1.1336 rank= 0.0281 kl= 0.1194
Val loss epoch 1000: reg= 5.9327 rank= 0.6286 kl= 0.1293
Regression R2 : 0.1377, Rank R2 : 0.7843
Experiment 2/20
lambda_reg=0.01, lambda_pair=2.0, margin_scale=0.3, gamma=0.01, beta=0.01, noise_std=0.001, alpha=0.0001,
encoder_lr=0.0001, feature_predictor_lr=0, cost_predictor_lr=0.01, seed=2001, use_feature=False
Train loss epoch 1000 : reg= 1.2274 rank= 0.0267 kl= 0.1169
Val loss epoch 1000: reg= 6.5162 rank= 0.6656 kl= 0.1265
Regression R2 : 0.1732, Rank R2 : 0.7900
Experiment 3/20
lambda_reg=0.01, lambda_pair=2.0, margin_scale=0.3, gamma=0.01, beta=0.01, noise_std=0.001, alpha=0.0001,
encoder_lr=0.0001, feature_predictor_lr=0, cost_predictor_lr=0.01, seed=2002, use_feature=False
Train loss epoch 1000 : reg= 1

KeyboardInterrupt: 

In [220]:
import torch

def recall_at_k(pred, labels, k=1):
    true_best_idx = torch.argmax(labels)
    topk_pred_idx = torch.topk(pred, k=k, largest=True).indices

    return float((topk_pred_idx == true_best_idx).any())



input_tensor = torch.from_numpy(input_data_scaled).float().to(device)
label_tensor = torch.from_numpy(costs).float().to(device)

vae_cost_model.eval()
with torch.no_grad():
    cost_pred, mean, logvar, z = vae_cost_model(input_tensor, use_mean=True)

k = 100
recall_score = recall_at_k(cost_pred, label_tensor, k=k)
print(f"Top-{k} Recall: {recall_score:.4f}")

Top-100 Recall: 0.0000


In [ ]:
# --- 하이퍼파라미터 설정 ---
# MAX_SEQ_LENGTH는 모든 코드 스니펫의 최대 extent 개수보다 커야 함
MAX_SEQ_LENGTH = max_len
HIDDEN_DIM = 256
LATENT_DIM = 64
BATCH_SIZE = 128
EPOCHS = 200
LEARNING_RATE = 1e-3


# 패딩 (MAX_SEQ_LENGTH로 길이 맞추기)
padded_data = np.ones((len(records["feature"]), MAX_SEQ_LENGTH), dtype=np.float32)
masks = np.zeros_like(padded_data, dtype=np.float32)
for i, ext_list in enumerate(records["feature"]):
    length = min(len(ext_list), MAX_SEQ_LENGTH)
    padded_data[i, :length] = ext_list[:length]
    masks[i, :length] = 1.0 # 실제 데이터가 있는 부분

data = torch.from_numpy(padded_data)
masks = torch.from_numpy(masks)

# --- [수정] 2. Log-Standardization (로그 표준화) ---

# 0이 없다고 가정했으므로 log(data) 바로 사용.
# (혹시 모를 0을 대비해 1e-8 더함)
data_log = torch.log(data + 1e-8) 

# 마스킹된 데이터(실제 값)만 사용하여 평균과 표준편차 계산
data_log_masked = data_log * masks
num_elements = masks.sum()

# (전체 데이터셋 기준)
data_mean = data_log_masked.sum() / num_elements
data_std = torch.sqrt( (((data_log_masked - data_mean) * masks)**2).sum() / num_elements )

print(f"Log Mean: {data_mean:.4f}, Log Std: {data_std:.4f}")

# (data_mean, data_std는 나중에 복원을 위해 저장)

# 표준화 (패딩된 0 영역은 (0-mean)/std가 되지만, 학습 시 손실에서 제외)
normalized_data = (data_log - data_mean) / (data_std)

# --- 3. DataLoader 준비 (마스크 포함) ---
dataset = TensorDataset(normalized_data, masks) # 마스크도 함께 전달
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# --- 4. 모델, 옵티마이저 초기화 (이전과 동일) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE(input_dim=MAX_SEQ_LENGTH, hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM, activation="relu").to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# --- 5. 학습 루프 (마스킹된 손실 계산) ---
model.train()
print("Training Start...")
for epoch in range(EPOCHS):
    epoch_loss = 0
    for (batch, mask) in dataloader: # 마스크도 함께 받음
        batch = batch.to(device)
        mask = mask.to(device)
        
        # Forward
        recon_batch, mean, logvar = model(batch * mask) # 입력 시에도 마스킹
        # recon_batch = model(batch * mask) # 입력 시에도 마스킹
        
        # [수정] 손실 계산 시 패딩 영역 제외 (마스크 곱하기)
        recon_error = F.mse_loss(recon_batch * mask, batch * mask, reduction='sum')
        KLD = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
        loss = recon_error + 0.5*KLD
        # loss = ae_loss(recon_batch * mask, batch * mask)
        # loss = vae_loss(recon_batch * mask, batch * mask)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(dataset)
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{EPOCHS}, Average Loss: {avg_loss:.4f}')

print("Training Complete.")

Log Mean: 1.1560, Log Std: 2.0146
Training Start...
Epoch 10/200, Average Loss: 7.2581
Epoch 20/200, Average Loss: 6.6265
Epoch 30/200, Average Loss: 6.2798
Epoch 40/200, Average Loss: 6.0641
Epoch 50/200, Average Loss: 6.0080
Epoch 60/200, Average Loss: 5.9396
Epoch 70/200, Average Loss: 5.8223
Epoch 80/200, Average Loss: 5.7692
Epoch 90/200, Average Loss: 5.6795
Epoch 100/200, Average Loss: 5.6905
Epoch 110/200, Average Loss: 5.6804
Epoch 120/200, Average Loss: 5.5932
Epoch 130/200, Average Loss: 5.6089
Epoch 140/200, Average Loss: 5.5722
Epoch 150/200, Average Loss: 5.5416
Epoch 160/200, Average Loss: 5.4812
Epoch 170/200, Average Loss: 5.4782
Epoch 180/200, Average Loss: 5.4997
Epoch 190/200, Average Loss: 5.4679
Epoch 200/200, Average Loss: 5.4695
Training Complete.


In [29]:
def preprocess(raw_data, max_len, data_mean, data_std):
    """원시 리스트를 모델 입력 텐서로 변환"""
    # 1. 패딩
    padded = np.zeros(max_len, dtype=np.float32)
    length = min(len(raw_data), max_len)
    padded[:length] = raw_data[:length]
    
    # 2. 텐서 변환
    tensor_data = torch.from_numpy(padded)
    
    # 3. 로그 변환 (0이 없다고 가정)
    log_data = torch.log(tensor_data + 1e-8) # 0 방지
    
    # 4. 표준화 (학습 시 사용한 평균, 표준편차 사용)
    normalized_data = (log_data - data_mean) / (data_std + 1e-8)
    
    # 5. 패딩 영역 0으로 마스킹 (중요)
    mask = torch.zeros_like(tensor_data)
    mask[:length] = 1.0
    
    return normalized_data.unsqueeze(0), mask.unsqueeze(0) # 배치 차원 추가

def postprocess(normalized_tensor, data_mean, data_std):
    """모델 출력 텐서를 원본 스케일의 extent로 변환"""
    # 1. 역-표준화
    log_tensor = (normalized_tensor * (data_std + 1e-8)) + data_mean
    
    # 2. 역-로그 (exp)
    extent_tensor = torch.exp(log_tensor)
    
    # 3. 정수로 변환 (extent는 정수)
    return torch.round(extent_tensor).int()




# --- 1. 설정 (학습 시와 동일하게) ---
MAX_SEQ_LENGTH = 45
HIDDEN_DIM = 256
LATENT_DIM = 128

MODEL_PATH = 'vae_model.pth'        # 학습된 모델 경로
STATS_PATH = 'norm_stats.pth'      # 학습 시 저장한 정규화 통계 (mean, std)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 2. 모델 및 통계 로드 ---
# (주의: 학습 시 아래와 같이 저장했다고 가정)
# torch.save(model, MODEL_PATH)
# torch.save({'mean': data_mean, 'std': data_std}, STATS_PATH)

try:
    # model = torch.load(MODEL_PATH, map_location=device)
    model.eval() # 테스트 모드로 설정

    # stats = torch.load(STATS_PATH)
    # data_mean = stats['mean'].to(device)
    # data_std = stats['std'].to(device)
except FileNotFoundError:
    print(f"오류: {MODEL_PATH} 또는 {STATS_PATH} 파일을 찾을 수 없습니다.")
    print("테스트 전에 모델과 통계 데이터를 먼저 학습/저장해야 합니다.")
    exit()

print(f"모델 {MODEL_PATH} 및 통계 {STATS_PATH} 로드 완료.\n")

# --- 3. 테스트 1: 재구성 (Reconstruction) ---
print("--- 1. 재구성 테스트 ---")
test_sample_raw = [28, 1, 112, 1, 1, 1, 37, 112, 1, 64, 112, 1, 16, 4, 2, 4, 2, 2, 2, 8, 2, 4]

# 전처리
test_input, mask = preprocess(test_sample_raw, MAX_SEQ_LENGTH, data_mean, data_std)
test_input = test_input.to(device)
mask = mask.to(device)

with torch.no_grad(): # 기울기 계산 비활성화
    recon_output, _, _ = model(test_input * mask)

# 후처리
recon_sample = postprocess(recon_output.squeeze(0), data_mean, data_std)

# 원본 길이만큼 잘라서 비교
original_len = len(test_sample_raw)

print(f" 원본 데이터: {test_sample_raw}")
print(f" 복원 데이터: {recon_sample[:original_len].tolist()}")

# (참고: VAE는 손실 압축이므로 원본과 완벽히 같지 않습니다)

# --- 4. 테스트 2: 생성 (Generation) ---
print("\n--- 2. 신규 생성 테스트 ---")
num_to_generate = 3

with torch.no_grad():
    # 잠재 공간(N(0,1))에서 무작위 샘플링
    z = torch.randn(num_to_generate, LATENT_DIM).to(device)
    
    # 디코더로 생성
    generated_output = model.decode(z)
    
# 후처리
generated_samples = postprocess(generated_output, data_mean, data_std)

print(f"{num_to_generate}개의 신규 루프 범위 벡터 생성:")
for i, sample in enumerate(generated_samples):
    # 0이나 음수는 1로 클리핑 (extent는 최소 1)
    cleaned_sample = torch.clamp(sample, min=1)
    
    # (편의상 앞 25개 정도만 출력)
    print(f" Sample {i+1}: {cleaned_sample[:25].tolist()}...")

모델 vae_model.pth 및 통계 norm_stats.pth 로드 완료.

--- 1. 재구성 테스트 ---
 원본 데이터: [28, 1, 112, 1, 1, 1, 37, 112, 1, 64, 112, 1, 16, 4, 2, 4, 2, 2, 2, 8, 2, 4]
 복원 데이터: [5, 4, 53, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 39, 24, 53, 1, 6]

--- 2. 신규 생성 테스트 ---
3개의 신규 루프 범위 벡터 생성:
 Sample 1: [3, 3, 272, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 74, 3, 270, 2, 1, 269, 2, 1]...
 Sample 2: [11, 4, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 30, 32, 10, 3, 28, 10, 1, 1]...
 Sample 3: [91, 3, 47, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 48, 47, 1, 7, 46, 2, 1]...
